In [ ]:

#importing important libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Stats & ML
from scipy import stats
from scipy.stats import chi2_contingency
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
   accuracy_score, precision_score, recall_score, f1_score,
   classification_report
)

# Plot settings (if running in notebook, uncomment)
# %matplotlib inline
sns.set(style="whitegrid", context="talk")
plt.rcParams["figure.dpi"] = 120

#  LOAD DATA

df = pd.read_csv(
   'adult.data',   # <-- replace with your file path
   header=None,
   names=[
       "age", "workclass", "fnlwgt", "education", "edyears",
       "marital-status", "occupation", "relationship", "race", "sex",
       "capital-gain", "capital-loss", "WorkHPweek", "country", "income"
   ],
   na_values=['?'],
   skipinitialspace=True
)

print("Initial shape:", df.shape)
display(df.sample(10))

# -------------------------------------------------------
# 2) MISSING VALUES HANDLING

print("\nMissing values before dropna():")
print(df.isna().sum())

df = df.dropna()

print("\nShape after dropna():", df.shape)
print("\nMissing values after dropna():")
print(df.isna().sum())

# -------------------------------------------------------
# 3) BASIC STRING CLEANING
# -------------------------------------------------------
strcolumns = df.select_dtypes(include='object').columns
for col in strcolumns:
   df[col] = df[col].astype(str).str.strip()

# Normalize 'country' naming variants
df["country"] = df["country"].replace({
   "United States": "US",
   "United-States": "US",
   "USA": "US"
})

display(df.head())

# -------------------------------------------------------
# 4) TYPE CASTING TO NUMERIC (where needed)
# -------------------------------------------------------
todonum = ["age", "fnlwgt", "edyears", "capital-gain", "capital-loss", "WorkHPweek"]
df[todonum] = df[todonum].apply(pd.to_numeric)
print("\nDtypes:\n", df[todonum].dtypes)

# -------------------------------------------------------
# 5) OUTLIER REMOVAL (domain caps + IQR)
# -------------------------------------------------------
# Domain-based sensible caps
df = df[(df["age"] >= 16) & (df["age"] <= 90)]
df = df[(df["WorkHPweek"] >= 1) & (df["WorkHPweek"] <= 100)]

# IQR-based filtering helper
def remove_iqr(dfin, col):
   Q1 = dfin[col].quantile(0.25)
   Q3 = dfin[col].quantile(0.75)
   IQR = Q3 - Q1
   low = Q1 - 1.5 * IQR
   high = Q3 + 1.5 * IQR
   return dfin[(dfin[col] >= low) & (dfin[col] <= high)]

for col in ["fnlwgt", "edyears", "capital-gain", "capital-loss"]:
   df = remove_iqr(df, col)

print("\nShape after outlier removal:", df.shape)

# -------------------------------------------------------
# 6) DROP IRRELEVANT/LESS USEFUL COLUMNS FOR THIS ANALYSIS
# -------------------------------------------------------
cols_to_drop = ["fnlwgt", "relationship", "country", "capital-gain", "capital-loss"]
df = df.drop(columns=cols_to_drop)
print("\nDropped columns:", cols_to_drop)
print("Shape after dropping columns:", df.shape)
display(df.head())

# -------------------------------------------------------
# 7) CREATE BINARY INCOME TARGET
# -------------------------------------------------------
df["income_bin"] = df["income"].apply(lambda x: 1 if ">50K" in x else 0)
display(df.head(1))

# -------------------------------------------------------
# 8) WEEKLY HOURS vs INCOME (Visualization + t-test)
# -------------------------------------------------------
plt.figure(figsize=(7, 5))
x = df[df.income_bin == 0]["WorkHPweek"]
y = df[df.income_bin == 1]["WorkHPweek"]
plt.boxplot([x, y], labels=["<=50K", ">50K"])
plt.title("Weekly Working Hours by Income Category")
plt.ylabel("Hours per week")
plt.yticks(np.arange(0, 101, 10))
plt.ylim(0, 100)
plt.grid(axis='y', linestyle='--', alpha=0.4)
plt.show()

tstat, pval = stats.ttest_ind(
   df[df.income_bin == 1]["WorkHPweek"],
   df[df.income_bin == 0]["WorkHPweek"],
   equal_var=False
)
print("\nT-test (Weekly Hours vs Income):")
print("t-statistic:", round(tstat, 4), "| p-value:", pval)

# -------------------------------------------------------
# 9) CORRELATION HEATMAP (numeric features)
# -------------------------------------------------------
num_for_corr = df[["age", "edyears", "WorkHPweek", "income_bin"]]
corr = num_for_corr.corr(method="pearson")
print("\nCorrelation matrix (numeric features):\n", corr)

plt.figure(figsize=(7, 6))
sns.heatmap(
   corr, annot=True, cmap="coolwarm",
   vmin=-1, vmax=1, linewidths=0.5, square=False
)
plt.title("Correlation Heatmap (Pearson)")
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# -------------------------------------------------------
# 10) EDUCATION vs INCOME
# -------------------------------------------------------
edu = df.groupby("edyears")["income_bin"].mean().reset_index()
plt.figure(figsize=(7, 5))
plt.plot(edu["edyears"], edu["income_bin"], marker="o")
plt.xlabel("Education (years)")
plt.ylabel("Probability of earning >50K")
plt.title("Education vs Earning Potential")
plt.grid(True, alpha=0.3)
plt.show()

edu_corr = df['edyears'].corr(df['income_bin'])
print("\nCorrelation (Education years vs Income_bin):", round(edu_corr, 4))

high_edu = df[df['edyears'] >= df['edyears'].median()]['income_bin']
low_edu = df[df['edyears'] < df['edyears'].median()]['income_bin']
tstat, pval = stats.ttest_ind(high_edu, low_edu, equal_var=False)
print("T-test (High vs Low Education): t-stat =", round(tstat, 4), ", p-value =", pval)

# -------------------------------------------------------
# 11) AGE vs INCOME (scatter)
# -------------------------------------------------------
plt.figure(figsize=(7, 5))
jitter = (np.random.rand(len(df)) - 0.5) / 10  # small jitter to visualize 0/1 spread
plt.scatter(df["age"], df["income_bin"] + jitter, alpha=0.25, s=10)
plt.xlabel("Age")
plt.ylabel("Income (0 or 1)")
plt.title("Age vs Earning Potential")
plt.grid(True, alpha=0.3)
plt.show()

# -------------------------------------------------------
# 12) *** NEW BLOCK *** CATEGORICAL vs INCOME ASSOCIATION
#     - Chi-square test of independence
#     - Cramér's V (strength of association)
#     - Bar charts of P(>50K) by category
# -------------------------------------------------------

# Select categorical columns (strings) excluding target 'income'
cat_cols = df.select_dtypes(include="object").columns.tolist()
cat_cols = [c for c in cat_cols if c != "income"]

assoc_results = []

def cramers_v_from_crosstab(ct):
   """
   Compute Cramér's V given a contingency table.
   V = sqrt(chi2 / (n * (min(k-1, r-1))))
   """
   chi2, p, dof, expected = chi2_contingency(ct)
   n = ct.values.sum()
   r, k = ct.shape
   denom = n * (min(r - 1, k - 1))
   if denom == 0:
       return np.nan, p, chi2
   V = np.sqrt(chi2 / denom)
   return V, p, chi2

print("\n=== Categorical Features: Chi-square & Cramér's V vs Income ===")
for c in cat_cols:
   # Build contingency table between category and income_bin
   ct = pd.crosstab(df[c], df["income_bin"])
   V, p, chi2 = cramers_v_from_crosstab(ct)
   assoc_results.append({"feature": c, "cramers_v": V, "p_value": p, "chi2": chi2})

   # Print quick summary
   print(f"{c:15s} | Cramér's V: {V:.4f} | p-value: {p:.3e} | chi2: {chi2:.2f} | levels: {ct.shape[0]}")

# Summarize associations sorted by strength
assoc_df = pd.DataFrame(assoc_results).sort_values(by="cramers_v", ascending=False)
print("\n\nCategorical–Income association (sorted by Cramér's V):")
display(assoc_df)

# Bar charts: Probability of >50K by category (sorted)
for c in cat_cols:
   prob = df.groupby(c)["income_bin"].mean().sort_values(ascending=False).reset_index()
   plt.figure(figsize=(10, 4 + 0.15 * prob.shape[0]))  # dynamic height if many categories
   sns.barplot(data=prob, x="income_bin", y=c, orient="h", palette="viridis")
   plt.xlabel("P(Income > 50K)")
   plt.ylabel(c)
   plt.title(f"P(>50K) by {c} (sorted)")
   plt.xlim(0, 1)
   plt.tight_layout()
   plt.show()

# -------------------------------------------------------
# 13) MODELING: Label Encode categorical columns for Logistic Regression
# -------------------------------------------------------
le = LabelEncoder()
for c in cat_cols:
   df[c] = le.fit_transform(df[c])  # convert str categories to numeric labels

print("\nCategorical columns encoded for modeling.")

# -------------------------------------------------------
# 14) TRAIN/TEST SPLIT & LOGISTIC REGRESSION
# -------------------------------------------------------
X = df.drop(columns=["income", "income_bin"])
y = df["income_bin"]

X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.25, random_state=42
)

model = LogisticRegression(max_iter=300)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("\n=== MODEL PERFORMANCE ===")
print("Accuracy :", round(accuracy_score(y_test, y_pred), 4))
print("Precision:", round(precision_score(y_test, y_pred), 4))
print("Recall   :", round(recall_score(y_test, y_pred), 4))
print("F1 Score :", round(f1_score(y_test, y_pred), 4))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# -------------------------------------------------------
# 15) FEATURE IMPORTANCE (Logistic Regression Coefficients)
# -------------------------------------------------------
feature_importance = pd.DataFrame({
   'Feature': X.columns,
   'Coefficient': model.coef_[0]
}).sort_values(by='Coefficient', ascending=False)

print("\nFeature Importance (Logistic Regression Coefficients):")
display(feature_importance)

# Highlight Weekly Working Hours coefficient
wh_coef = feature_importance.loc[feature_importance['Feature'] == 'WorkHPweek', 'Coefficient']
if not wh_coef.empty:
   print("\nWeekly Working Hours Coefficient:", float(wh_coef.values[0]))

# Visualize coefficients
plt.figure(figsize=(8, max(6, 0.35 * len(feature_importance))))
plt.barh(feature_importance['Feature'], feature_importance['Coefficient'], color='steelblue')
plt.title("Feature Importance (Logistic Regression Coefficients)")
plt.xlabel("Coefficient Value")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()
 

